# Real NVP （CIFAR10）

In [ ]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 32
epochs = 5
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda:1"
else:
    device = "cpu"

In [ ]:
root = '../data'
num_workers = 8

transform_train = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_loader = DataLoader(datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train),
                         batch_size=batch_size, shuffle=True, num_workers=num_workers)

test_loader = DataLoader(datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test),
                        batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
from pixyz.distributions import Normal, InverseTransformedDistribution
from pixyz.flows import AffineCoupling, FlowList, Squeeze, Unsqueeze, Preprocess, Flow
from pixyz.layers import ResNet
from pixyz.models import ML

In [ ]:
# prior model p(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=1, name="p_prior")

In [ ]:
class ScaleTranslateNet(nn.Module):
    def __init__(self, in_channels, mid_channels):
        super().__init__()
        self.resnet = ResNet(in_channels=in_channels, mid_channels=mid_channels, out_channels=in_channels*2,
                             num_blocks=8, kernel_size=3, padding=1,
                             double_after_norm=True)

    def forward(self, x):
        s_t = self.resnet(x)
        log_s, t = torch.chunk(s_t, 2, dim=1)
        log_s = torch.tanh(log_s)
        return log_s, t

In [ ]:
in_channels = 3
mid_channels = 64
num_scales = 2

flow_list = []

flow_list.append(Preprocess())

# Coupling_Layer(checkboard) x3
for i in range(3):    
    flow_list.append(AffineCoupling(in_features=in_channels, mask_type="checkerboard",
                                    scale_translate_net=ScaleTranslateNet(in_channels, mid_channels),
                                    inverse_mask=(i%2!=0)))

# Squeeze -> 3x coupling (channel-wise)
flow_list.append(Squeeze())

for i in range(3):
    flow_list.append(AffineCoupling(in_features=in_channels*4, mask_type="channel_wise",
                                    scale_translate_net=ScaleTranslateNet(in_channels*4, mid_channels*2),
                                    inverse_mask=(i%2!=0)))
flow_list.append(Unsqueeze())

f = FlowList(flow_list)

In [ ]:
# inverse transformed distribution (z -> f^-1 -> x)
p = InverseTransformedDistribution(prior=prior, flow=f, var=["x"])
p.to(device)

In [ ]:
model = ML(p, optimizer=optim.Adam, optimizer_params={"lr":1e-3})
print(model)

In [ ]:
def train(epoch):
    train_loss = 0
    
    for x, _ in tqdm(train_loader):
        x = x.to(device)
        loss = model.train({"x": x})
        train_loss += loss

    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [ ]:
def test(epoch):
    test_loss = 0
    for x, _ in test_loader:
        x = x.to(device)
        loss = model.test({"x": x})
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

In [ ]:
def plot_image_from_latent(z_sample):
    with torch.no_grad():
        sample = p.inverse(z_sample).cpu()
        return sample
    
def plot_reconstrunction(x):
    with torch.no_grad():
        z = p.forward(x, compute_jacobian=False)
        recon_batch = p.inverse(z)
    
        comparison = torch.cat([x.view(-1, 3, 32, 32), recon_batch]).cpu()
        return comparison

In [ ]:
writer = SummaryWriter()

z_sample = torch.randn(64, 3, 32, 32).to(device)
_x, _ = iter(test_loader).next()
_x = _x.to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    recon = plot_reconstrunction(_x[:8])
    sample = plot_image_from_latent(z_sample)

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      
    
    writer.add_images('Image_from_latent', sample, epoch)
    writer.add_images('Image_reconstrunction', recon, epoch)
    
writer.close()